<a href="https://colab.research.google.com/github/AndreAilton/Sistema_Bancario/blob/main/Sistema_Bancario(POO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importações Bibliotecas**

In [ ]:
from datetime import datetime
import pytz
from random import randint
import requests
import copy

# **Agencia**

In [ ]:
class Agencia:
  def __init__(self, telefone, cnpj, numero):
      self.telefone = telefone
      self.cnpj = cnpj
      self.numero = numero
      self.caixa = 10000
      self.emprestimos = []
      self.clientes = []
      self.lucro = []
      self.Historico_Emprestimo = []
      self.cartao = []

  @staticmethod
  def _data_hora():
      fuso_Br = pytz.timezone('Brazil/East')
      horario = datetime.now(fuso_Br)
      return horario.strftime('%d-%m-%Y  %H:%M:%S')

  def verificar_caixa(self):
      if self.caixa < 10000:
          print(f'Caixa Abaixo do nível recomendado, seu valor atual é de {self.caixa}')
      else:
          print(f'Caixa está OK, seu valor atual é de: {self.caixa}')

  def verificar_emprestimos(self):
      print(self.emprestimos)
      lucro = sum([l[2] - l[1] for l in self.emprestimos])
      print(f'Foram feitos {len(self.emprestimos)} empréstimos com lucro estimado de {lucro}R$')

  def depositar_caixa(self, valor):
      self.caixa += valor

  def _get_taxa_selic(self):
      url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.4390/dados/ultimos/1?formato=json'
      try:
          response = requests.get(url)
          data = response.json()
          taxa_selic = data[0]['valor']
          return taxa_selic
      except:
          print('Erro ao obter taxa Selic')
          return None

  def solicitar_emprestimo(self, cliente, valor, meses):
      taxa_selic = self.get_taxa_selic()
      if taxa_selic is not None:
          emprestimo = (valor * (meses * taxa_selic))
          self.emprestimos.append((cliente,valor,emprestimo, taxa_selic))
          self.caixa -= valor
          cliente.Historico_Movimentacao.append((f'Emprestimo:{valor}',
                                                 self._data_hora(),
                                                 f'Pagar: em {meses}',
                                                 f'Valor Total {valor *(taxa_selic * meses)}'))
          print(f'Empréstimo de {valor}R$ Realizado Com sucesso para {cliente}')
          print(f'Taxa Selic atual: {taxa_selic}')

      else:
          print('Não foi possível solicitar o empréstimo')

  def realizar_pagamento(self, cliente, valor):
      for emprestimo in self.emprestimos:
          if emprestimo[0] == cliente:
              if valor >= emprestimo[1]:
                  self.caixa += emprestimo[1]
                  self.emprestimos.remove(emprestimo)
                  self.Historico_Emprestimo.append(emprestimo)
                  print(f'Pagamento de {emprestimo[1]}R$ realizado por {cliente}')
              else:
                  print('Valor insuficiente para realizar o pagamento')
              break
      else:
          print(f'Não foi encontrado empréstimo pendente para {cliente}')

  def adicionar_cliente(self, Nome, Cpf,dataNascimento, telefone):
      self.clientes.append((Nome, Cpf,dataNascimento, telefone))
      print(f'Cliente {Nome} adicionado com sucesso')

  def remover_cliente(self,  Nome, Cpf,dataNascimento, telefone):
      if (Nome, Cpf,dataNascimento, telefone) in self.clientes:
          self.clientes.remove((Nome, Cpf,dataNascimento, telefone))
          print(f'Cliente {Nome} removido com sucesso')
      else:
          print(f'Cliente {Nome} não encontrado')

  def adicionar_cartao(self, conta_corrente, cartao):
      cliente_cartao = (conta_corrente, cartao)
      self.cartao.append(cliente_cartao)
      print(f'Cartão {cartao} adicionado para o cliente {conta_corrente}')

  def remover_cartao(self, conta_corrente, cartao):
      for cliente_cartao in self.cartao:
          if cliente_cartao[0] == conta_corrente and cliente_cartao[1] == cartao:
              self.cartao.remove(cliente_cartao)
              print(f'Cartão {cartao} removido para o cliente {conta_corrente}')
              break
      else:
          print(f'Cartão {cartao} não encontrado para o cliente {conta_corrente}')


# **Conta Corrente**

In [ ]:
class Contacorrente:
    def __init__(self, nome, cpf, Agencia):
        for i, (N, C, Dn, t) in enumerate(Agencia.clientes):
            if (i, nome, cpf) == (i, N, C):
                self.saldo = 0
                self.nome = nome
                self.cpf = cpf
                self.data_nascimento = Dn
                self.Historico_Movimentacao = []
                self.Historico_Ted = []
                self.numero = randint(1000, 9999)
                self.agencia = Agencia
                self.cartao = []
                print('Contacorrente criada com sucesso')
            else:
              pass
            
    @staticmethod
    def _data_hora():
        fuso_Br = pytz.timezone('Brazil/East')
        horario = datetime.now(fuso_Br)
        return horario.strftime('%d-%m-%Y  %H:%M:%S')

    @staticmethod
    def _inputSN(metodo):
        print(f'Deseja {metodo} S/N ?')
        resposta = None
        while resposta not in ('N', 'S'):
            resposta = input()
            resposta = resposta.upper()
            if resposta not in ('N', 'S'):
                print("Opção inválida. Tente novamente.")
        return resposta
    
    @staticmethod
    def _impressao(imprimir):
        print(f"{'=' * 54}\n{'=' * 15 + 'COMPROVANTE DE IMPRESSÃO' + '=' * 15}\n{'=' * 54}\n{Contacorrente._data_hora()}\n{'-' * 54}")
        if imprimir == ' ':
          return 'Historico'
        else:
          for c in imprimir:
            print(c)
          return ' '


    def ConsultarHTED(self):
        print('Aqui está seu histórico de transferências bancárias')
        print(self.Historico_Ted)
        resposta = self._inputSN('Imprimir')
        if resposta == 'S':
            for c in self.Historico_Ted:
                print(c)

    def Consultarhistorico(self):
        print('Aqui está seu histórico de transações bancárias')
        valor = 0
        contador = 0
        for c, x in self.Historico_Movimentacao:
            if 'Deposito' in c:
                valor += int(c[9:])
                contador += 1
        print(f'Foram realizados {contador} depósitos que somam o valor de {valor}')
        imprimir = f'Foram realizados {contador} depósitos que somam o valor de {valor}\n'
        valor = 0
        contador = 0
        for c, x in self.Historico_Movimentacao:
            if 'Saque' in c:
                valor += int(c[6:])
                contador += 1
        print(f'Foram realizados {contador} Saques que somam o valor de {valor}')
        imprimir += f'Foram realizados {contador} Saques que somam o valor de {valor}\n'
        valor = 0
        contador = 0

        for a,b,c in self.Historico_Ted:
            if 'transferencia' in a:
                valor1 = a.split(':')[1]
                valor += int(valor1)
                contador += 1
        print(f'Foram realizadas {contador} transferências que somam o valor de {valor}')
        imprimir += f'Foram realizadas {contador} transferências que somam o valor de {valor}\n'
        resposta = self._inputSN('Imprimir')

        if resposta == 'S':
          print(Contacorrente._impressao(' '))
          for c, x in self.Historico_Movimentacao:
            if 'Deposito' in c:
              print(c)
            if 'Saque' in c:
              print(c)
          for a,b,c in self.Historico_Ted:
            if 'transferencia' in a:
              print(a)

    def ConsultarSaldo(self):
        resposta = self._inputSN('Consultar saldo?')
        if resposta == 'S':
            print('Seu saldo atual é de {}'.format(self.saldo))

    def Depositar(self, valor):
        resposta = self._inputSN(f'Depositar R${valor}')
        if resposta == 'S':
            self.saldo += valor
            self.Historico_Movimentacao.append((f'Deposito:{valor}', self._data_hora()))
        else:
            print('Insira um novo valor')
            self.Depositar(int(input()))

    def Sacar(self, valor):
        resposta = self._inputSN(f'Sacar R${valor}')
        if self.agencia.caixa > valor:
            if resposta == 'S':
                if self.saldo - valor < 0:
                    print('Você não consegue sacar este valor!')
                    self.ConsultarSaldo()
                else:
                    self.saldo -= valor
                    self.Historico_Movimentacao.append((f'Saque:{valor}', self._data_hora()))
                    self.agencia.caixa -= valor
                    print('Saque realizado com Sucesso!')
                    resposta = self._inputSN(f'Realizar um novo saque?')
            if resposta == 'S':
                print('Insira Novo valor de saque')
                self.Sacar(int(input()))
        else:
            print('Desculpe, estamos sem caixa. Favor retornar em algum outro momento.')

    def TED(self, valor, conta_destino):
        resposta = self._inputSN(f'Deseja realizar uma transferência no valor de R${valor}')
        if resposta == 'S':
            if self.saldo - valor < 0:
                print('Você não consegue transferir este valor!')
                self.ConsultarSaldo()
            else:
                self.saldo -= valor
                self.Historico_Ted.append((f'transferencia:{valor}',
                                           ({conta_destino.nome}, f'***{conta_destino.cpf[4:7]}***'),
                                           self._data_hora()))
                conta_destino.saldo += valor
                conta_destino.Historico_Ted.append((f'transferencia:{valor}', self._data_hora()))
        resposta = self._inputSN(f'Realizar nova transferência para {conta_destino.nome}?')
        if resposta == 'S':
            print('Insira um novo valor')
            self.TED(int(input()), conta_destino)


# **Cartão de credito**

In [ ]:
class CartaoCredito:
    @staticmethod
    def _data_hora(horario=0):
        if horario == 0:
            fuso_Br = pytz.timezone('Brazil/East')
            horario = datetime.now(fuso_Br)
            return "{}/{}".format(horario.month, horario.year - 2000 + 4)
        elif horario == 1:
            fuso_Br = pytz.timezone('Brazil/East')
            horario = datetime.now(fuso_Br)
            return horario.strftime('%d-%m-%Y  %H:%M:%S')
        elif horario == 2:
            fuso_Br = pytz.timezone('Brazil/East')
            horario = datetime.now(fuso_Br)
            return "{}/{}/{}".format(horario.day, horario.month, horario.year)

    @staticmethod
    def _numero_mes(mes):
        meses = [
            'lista', 'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho',
            'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'
        ]
        mes = meses[mes]
        return mes

    @staticmethod
    def _converter_data(data):
        data = int(data.split('/')[1])
        return data

    @staticmethod
    def _input_sn(metodo):
        print(f'Deseja {metodo} S/N ?')
        resposta = None
        while resposta not in ('N', 'S'):
            resposta = input()
            resposta = resposta.upper()
            if resposta not in ('N', 'S'):
                print("Opção inválida. Tente novamente.")
        return resposta

    @staticmethod
    def _input_int(metodo):
        while True:
            valor = input(metodo)
            if valor.isdigit():
                return int(valor)
            else:
                print("Entrada inválida. Tente novamente.")

    def __init__(self, conta_corrente, nome_cartao='Cartao'):
        self.nome = conta_corrente.nome
        self.conta_corrente = conta_corrente

        if nome_cartao == 'Cartao':
            self.nome_cartao = nome_cartao + str(len(conta_corrente.cartao) + 1)
        else:
            self.nome_cartao = nome_cartao
        self.num_cartao = randint(100000000000, 999999999999)
        self.num_agencia = conta_corrente.agencia.numero
        self.data = CartaoCredito._data_hora()
        self.ccv = randint(100, 999)
        self.limite_max = 5000
        self.limite_ajuste = 1000
        self.historico = []
        self.historico_fatura = []
        conta_corrente.cartao.append((nome_cartao, self))
        conta_corrente.agencia.cartao.append((nome_cartao, self))

    def _verificar_limite(self):
        return self.limite_ajuste

    def _verificar_limite_atual(self):
        return self.limite_ajuste - self._verificar_fatura()

    def _verificar_fatura(self):
        soma = 0
        if self.historico_fatura == []:
            return soma
        else:
            fatura = copy.deepcopy(self.historico_fatura)
            for i, (c, b) in enumerate(fatura):
                x = CartaoCredito._converter_data(b)
                fatura[i][1] = x
            indice1 = fatura[0][1]
            for i, (c, b) in enumerate(fatura):
                soma += c
            return soma

    def ajustar_limite(self):
        print(f'Seu limite atual é de R${self.limite_ajuste}, limite disponível R$ {self.limite_max - self.limite_ajuste}')
        if CartaoCredito._input_sn('Ajustar o seu Limite') == 'S':
            while True:
                valor = CartaoCredito._input_int('Insira o novo valor do limite: ')
                if valor >= self._verificar_fatura():
                    if valor <= self.limite_max:
                        self.limite_ajuste = valor
                        break
                    else:
                        print(f'Você não possui este valor, limite disponível R$ {self.limite_max - self.limite_ajuste}')
                else:
                    print(f'Valor maior que sua fatura atual de {self._verificar_fatura()}')

    def pagar_fatura(self):
        if self._verificar_fatura() < self.conta_corrente.saldo:
          if self._verificar_fatura() == 0:
            print('Voce ja pagou a fatura atual')
          else:
              self.conta_corrente.saldo = self.conta_corrente.saldo - self._verificar_fatura()
              soma = 0
              fatura = copy.deepcopy(self.historico_fatura)
              for i, (c, b) in enumerate(fatura):
                  x = CartaoCredito._converter_data(b)
                  fatura[i][1] = x
              indice1 = fatura[0][1]
              for i, (c, b) in enumerate(fatura):
                  if indice1 == b:
                      del self.historico_fatura[i]
              self.conta_corrente.Historico_Movimentacao.append((f'Fatura:{self._verificar_fatura()}', self._data_hora(1)))
              print('Fatura atual paga com sucesso')
        else:
              print(f'Saldo em ContaCorrente indisponível. Saldo atual R$ {self.conta_corrente.saldo}.'
                    f'Favor depositar {self._verificar_fatura() - self.conta_corrente.saldo}')
              print(f'Fatura atual de R$ {self._verificar_fatura()}')

    def comprar(self, loja, valor, num_cartao, num_agencia, ccv):
        resultado = False
        for i,(c, x) in enumerate(self.conta_corrente.agencia.cartao):
            if (x.num_cartao, int(x.num_agencia), x.ccv) == (num_cartao, num_agencia, ccv):
                if valor <= self._verificar_limite_atual():
                    print('Transação realizada com sucesso')
                    self.historico.append((f'Compra realizada em {loja}', f'data {CartaoCredito._data_hora(1)}',
                                           f'no valor R$:{valor}'))
                    self.historico_fatura.append([valor, CartaoCredito._data_hora(2)])
                    resultado = True
                else:
                    print('Transação recusada. Você não possui limite para esta transação')
                    resultado = True
        
        
        if not resultado:
            print('Cartão não registrado no banco de dados. Verifique os números')
